# First Steps with HybridAGI

In this notebook, we will guide you through the basic concepts and features of HybridAGI, a neuro-symbolic agent system that allows you to program its behavior using a graph-based prompt programming approach.

In this notebook, we will cover the following topics:

- Introduction to HybridAGI Graph Prompt Programming
- Installation and setup of HybridAGI
- Overview of the Domain Specific Language (DSL) of HybridAGI and its basic syntax
- Examples of graph-based prompt programming using the DSL

By the end of this notebook, you will have an understanding of the basic concepts of HybridAGI and be able to start experimenting with it on your own.

## Setting up the Knowledge Base locally

HybridAGI works with a low-latency hybrid vector/graph database called [FalkorDB](https://www.falkordb.com/). This knowledge base should be started when you work with HybridAGI.

Start the database using docker with the following command in you terminal:

```bash
docker run -p 6379:6379 -p 3000:3000 -it --rm falkordb/falkordb:edge
```

## Setting up Ollama to test your pipelines locally

To test your pipelines locally, we need to setup Ollama to run the LLMs in a separate server. 

### Linux:
Open a terminal and use the following commands:

```bash
# Install it on linux
curl -fsSL https://ollama.com/install.sh | sh
# Then launch the Ollama server with
ollama pull mistral
ollama serve
```

### Windows:
Downloads the Ollama executable [here](https://ollama.com/download/OllamaSetup.exe)

In [8]:
# Install HybridAGI package in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install git+https://github.com/SynaLinks/HybridAGI.git
!{sys.executable} -m pip install ..

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing /home/yoan/Workspace/HybridAGI
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hybridagi: filename=hybridagi-0.2.3-py3-none-any.whl size=111987 sha256=c0fa1392aa6317c3ab923ab83e593f79955a8c2ff363a6238428211c87b87a53
  Stored in directory: /tmp/pip-ephem-wheel-cache-2hhkaiuf/wheels/89/e0/7c/3879c93b02bba3dbfbb0662e1d980e95abc68bfe985dbda52d
Successfully built hybridagi
  Attempting uninstall: hybridagi
    Found existing installation: hybridagi 0.2.3
    Uninstalling hybridagi-0.2.3:
      Successfully uninstalled hybridagi-0.2.3

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


### Graph-based prompt programs introduction

HybridAGI is an agent system that introduces a `GraphProgramInterpreter`. Similar to how the Python interpreter executes a program line by line, HybridAGI's interpreter executes a graph program node by node. To store these graph programs, we use a hybrid vector/graph database. 

Additionally, a dependency tree is automatically generated when loading the programs. This dependency tree enables the system to protect the main prompting mechanism and its dependencies, as well as allowing similarity-based search over the Agent's graph programs.

In the following example, we have a simple graph program consisting of three nodes: `start`, `end`, and `answer`. The `start` and `end` nodes are control nodes that denote the beginning and end of the program, respectively. The `answer` node is an action node that uses the `Speak` tool to answer the objective's question.

The edges between the nodes are defined using the `-[:NEXT]->` syntax, which indicates that the interpreter should move to the next node in the program. In this case, the interpreter will start at the `start` node, move to the `answer` node, and then finally move to the `end` node.

In [9]:

import dspy
from hybridagi import HybridAGI

# Setup the LM
lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])

dspy.configure(lm=lm)

agent = HybridAGI(
    agent_name = "first_steps",
)

agent.add_programs_from_folders(["programs/first_steps"])

# Now let's try our pipeline
prediction = agent.execute("What is the city of the spacial and robotics industry in France?")

print(prediction.final_answer)

The city of Toulouse is known for its significant contributions to the spatial and robotics industry in France.


In [10]:
# We can also check the program trace

print(prediction.program_trace)

 --- Step 0 ---
Call Program: main
Program Purpose: What is the city of the spacial and robotics industry in France?
 --- Step 1 ---
Action Purpose: Answer the objective's question
Action: {
  "message": "The city of Toulouse is known for its significant contributions to the spatial and robotics industry in France."
}
 --- Step 2 ---
End Program: main


In [11]:
# Or the chat history

print(prediction.chat_history)

[
  {
    "role": "User",
    "message": "What is the city of the spacial and robotics industry in France?"
  },
  {
    "role": "AI",
    "message": "The city of Toulouse is known for its significant contributions to the spatial and robotics industry in France."
  }
]
